# Use llama (nomic_embed-text) in Docker to extract embeddings
Step 1: Start the Ollama Docker container (if not already running)

docker run -d \
  --gpus all \
  --name ollama \
  -v ~/.ollama:/root/.ollama \
  -p 11434:11434 \
  ollama/ollama

Pull the embedding model (e.g., nomic-embed-text)

docker exec -it ollama ollama pull nomic-embed-text

Step 2: run the following code plock

In [1]:
import pandas as pd
import requests

# Load CSV file
df = pd.read_csv("data.csv")
sentences = df["text"].tolist()  # Replace "text" with your column name

# Ollama API endpoint
OLLAMA_HOST = "http://localhost:11434"
MODEL_NAME = "nomic-embed-text"  # Or any embedding model (e.g., "llama2")

# Generate embeddings
embeddings = []
for sentence in sentences:
    response = requests.post(
        f"{OLLAMA_HOST}/api/embeddings",
        json={
            "model": MODEL_NAME,
            "prompt": sentence
        }
    )
    if response.status_code == 200:
        embedding = response.json()["embedding"]  # Typo in Ollama's response? Double-check the key!
        embeddings.append(embedding)
    else:
        print(f"Failed for: {sentence}")
        embeddings.append(None)

# Add embeddings to DataFrame
df["embedding"] = embeddings

# Save to new CSV
df.to_csv("data_with_embeddings-ollama.csv", index=False)

# Use deepseek in Docker to extract embeddings
Step 1: Start Ollama in Docker

Start Ollama with GPU support

docker run -d \
  --gpus all \
  --name ollama \
  -v ~/.ollama:/root/.ollama \
  -p 11434:11434 \
  ollama/ollama

Pull your desired model (e.g., DeepSeek):

docker exec -it ollama ollama pull deepseek-r1:8b

tep 2: Then run the following code:

In [ ]:
import pandas as pd
import requests

# Load CSV data
df = pd.read_csv("data.csv")
sentences = df["text"].tolist()

# Ollama API endpoint
OLLAMA_HOST = "http://localhost:11434"

# Generate embeddings
embeddings = []
for sentence in sentences:
    response = requests.post(
        f"{OLLAMA_HOST}/api/embeddings",
        json={
            "model": "deepseek-r1:8b",  # Your model name
            "prompt": sentence
        }
    )
    if response.status_code == 200:
        embedding = response.json().get("embedding")
        embeddings.append(embedding)
    else:
        print(f"Error: {response.text}")
        embeddings.append(None)

# Save results
df["embedding"] = embeddings
df.to_csv("data_with_embedding_deepseek.csv", index=False)

# NOT USE: Get Embeddings From Deepseek R1 With vLLM
source: https://www.youtube.com/watch?v=OMzZ5SABapA
- Note that DeepSeek currently does not offer any embeddings models, this method is not validated.
- Current server's CUDA 11.4 is outdated: PyTorch 1.10.0 + CUDA 11.3 will not work with modern libraries like vLLM.
    - Solution: need admin to upgrade CUDA.

In [ ]:
# !pip install vllm

In [15]:
torch.cuda.is_available()

False

In [10]:
import pandas as pd
import torch
# Load CSV file
df = pd.read_csv("data.csv")
sentences = df["text"].tolist()  # Replace "text" with your column name

from vllm import LLM
modelds = LLM(
    model = 'deepseek-ai/DeepSeek-R1-Distill-Llama-8B',
    task = 'embed',
    enforce_eager = True
)
output = model.embed(sentences)

INFO 02-20 11:31:23 __init__.py:207] Automatically detected platform cuda.


ImportError: /home/local/PSYCH-ADS/xuqian_chen/anaconda3/envs/jupyter_env/envs/neuronlp/lib/python3.10/site-packages/vllm/_C.abi3.so: undefined symbol: cuTensorMapEncodeTiled